In [12]:
%matplotlib inline

import os
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

DATA_RAW = Path(os.getcwd()) / "data" / "raw" / "anomaly"
DATA_FINAL = Path(os.getcwd()) / "data" / "final" / "anomaly"

# Series1 (Train)

This is all of the consumption data for the meters in the buildings.

 - Remove 'normal_abnormal_tag' column
 - Write as properly comma separated

In [18]:
dfs = []

for f in (DATA_RAW / 'Series_Given1').glob('*.csv'):
    dfs.append(pd.read_csv(f, parse_dates=['Timestamp'], sep=';'))
    
given1 = pd.concat(dfs).sort_values(['meter_id', 'Timestamp'])

print(given1.shape)
given1.head()

(43668606, 4)


,meter_id,Timestamp,Values,normal_abnormal_tag
2532,2,2015-06-11 00:00:00,2035.0,NaN
2543,2,2015-06-11 00:15:00,2074.0,NaN
2544,2,2015-06-11 00:30:00,2062.0,NaN
2525,2,2015-06-11 00:45:00,2025.0,NaN
2534,2,2015-06-11 01:00:00,2034.0,NaN


In [31]:
# check if overlap with consumption challenge
consumption = pd.read_csv(Path("data/final/consumption/public/train.csv"), index_col=0)

consumption.head()


,obs_id,SiteId,Timestamp,Value
0,744519,1,2014-09-03 00:00:00,9.096555e+05
1,7627564,1,2014-09-04 00:00:00,1.748273e+06
2,7034705,1,2014-09-05 00:00:00,NaN
3,5995486,1,2014-09-06 00:00:00,NaN
4,7326510,1,2014-09-07 00:00:00,NaN


In [34]:

sites = consumption.SiteId.unique()

In [38]:
meters = given1.meter_id.unique()
meters

in_both = set()
for m in meters:
    if isinstance(m, str):
        m = int(m.split("_")[0])
        
    if m in sites:
        in_both.add(m)
        
in_both

{2, 38, 234}

In [62]:
given1.drop(['normal_abnormal_tag'], axis=1, inplace=True)

In [63]:
given1.to_csv(DATA_FINAL / "public" / "train.csv")

# Test Set (Given2, Secret)

 - All are contained in Secret, no need to use Given2

In [45]:
dfs = []

for f in (DATA_RAW / 'Series_Secret').glob('*.csv'):
    dfs.append(pd.read_csv(f, parse_dates=['Timestamp'], sep=';'))
    
secret = pd.concat(dfs).sort_values(['meter_id', 'Timestamp'])

print(secret.shape)
secret.head()

(402971, 6)


,meter_id,Timestamp,Values,site_id,normal_abnormal_tag,given
0,234_203,2013-11-03 00:00:00,NaN,203,normal,secret
1,234_203,2013-11-03 00:30:00,NaN,203,normal,secret
2,234_203,2013-11-03 01:00:00,NaN,203,normal,secret
3,234_203,2013-11-03 01:30:00,NaN,203,normal,secret
4,234_203,2013-11-03 02:00:00,NaN,203,normal,secret


In [46]:
secret.meter_id.value_counts()

38_9686    254547
334_61      75224
234_203     73200
Name: meter_id, dtype: int64

In [47]:
secret.normal_abnormal_tag.value_counts()

normal                                                                                                 369778
day with abnormal consumption                                                                           25283
abnormal consumption regarding the outside temperature                                                   4745
abnormal consumption during unoccupied hours                                                             2255
day with abnormal consumption+abnormal consumption regarding the outside temperature                      672
abnormal consumption during unoccupied hours+abnormal consumption regarding the outside temperature       199
punctual anomaly                                                                                           39
Name: normal_abnormal_tag, dtype: int64

In [48]:
secret.meter_id.isin(given1.meter_id.unique()).all()

True

In [49]:
pd.isnull(secret.Values).sum()

336

In [50]:
secret.dropna(inplace=True)

In [51]:
rng = np.random.RandomState(900)
idx = rng.permutation(np.arange(secret.shape[0]))
idx

secret.index = pd.Series(idx, name='obs_id')

secret.head()

,meter_id,Timestamp,Values,site_id,normal_abnormal_tag,given
obs_id,,,,,,
171227,234_203,2013-11-03 19:00:00,11636.0,203,normal,secret
177020,234_203,2013-11-03 19:30:00,11106.0,203,normal,secret
165375,234_203,2013-11-03 20:00:00,10957.0,203,normal,secret
207176,234_203,2013-11-03 20:30:00,11157.0,203,normal,secret
353253,234_203,2013-11-03 21:00:00,12196.0,203,normal,secret


In [52]:
secret['is_normal'] = (secret.normal_abnormal_tag == 'normal')

In [55]:
test = secret[['meter_id', 'Timestamp', 'is_normal']]

submission_format = test.copy()
submission_format.is_normal = True

public_subset = (secret.given != 'secret').to_frame()

In [56]:
test.head()

,meter_id,Timestamp,is_normal
obs_id,,,
171227,234_203,2013-11-03 19:00:00,True
177020,234_203,2013-11-03 19:30:00,True
165375,234_203,2013-11-03 20:00:00,True
207176,234_203,2013-11-03 20:30:00,True
353253,234_203,2013-11-03 21:00:00,True


In [57]:
submission_format.head()

,meter_id,Timestamp,is_normal
obs_id,,,
171227,234_203,2013-11-03 19:00:00,True
177020,234_203,2013-11-03 19:30:00,True
165375,234_203,2013-11-03 20:00:00,True
207176,234_203,2013-11-03 20:30:00,True
353253,234_203,2013-11-03 21:00:00,True


In [58]:
public_subset.head()

,given
obs_id,
171227,False
177020,False
165375,False
207176,False
353253,False


In [61]:
submission_format.to_csv(DATA_FINAL / "public" / "submission_format.csv")
test.to_csv(DATA_FINAL / "private" / "test.csv")
public_subset.to_csv(DATA_FINAL / "private" / "public_subset.csv")

# Metadata

 - Use comma separated instead of semicolon

In [70]:
meta_data = pd.read_csv(DATA_RAW / 'Metadata' / 'meter-meta.csv', sep=';')
meta_data.to_csv(DATA_FINAL / "public" / "metadata.csv")

# Holidays

In [71]:
holidays = pd.read_csv(DATA_RAW / 'Holidays' / 'holidays.csv', sep=';')
holidays.to_csv(DATA_FINAL / "public" / "holidays.csv")

# Weather

 - Coalese into single data frame

In [73]:
dfs = []

for f in (DATA_RAW / 'Weather').glob('*.csv'):
    dfs.append(pd.read_csv(f, sep=';', parse_dates=['Timestamp']))
    
weather = pd.concat(dfs).sort_values(['site_id', 'Timestamp'])

print(weather.shape)

(391628, 4)


In [74]:
weather.head()

,Timestamp,Temperature,Distance,site_id
42628,2012-01-01 01:00:00,3.9,11.902932,38
42629,2012-01-01 02:00:00,4.1,11.902932,38
42630,2012-01-01 03:00:00,4.2,11.902932,38
42631,2012-01-01 04:00:00,4.1,11.902932,38
42632,2012-01-01 05:00:00,4.3,11.902932,38


In [75]:
weather.to_csv(DATA_FINAL / "public" / "weather.csv")